In [40]:
import os
import cv2
import re
import numpy as np
from PIL import Image
from numpy.fft import fft2, ifft2

Image.MAX_IMAGE_PIXELS = None

# --- Utility functions ---
def list_files_with_pattern(directory_path, file_pattern):
    file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file_pattern in file:
                file_paths.append(os.path.join(root, file))
    return file_paths


def extract_group_name(file_path):
    """Extract group name matching pattern 'T0XX' from file path"""
    match = re.search(r'T0\d{2}', file_path)
    return match.group(0) if match else None

def get_unique_group_names(file_paths):
    """Get sorted list of unique group names from file paths"""
    group_names = set()
    for path in file_paths:
        group_name = extract_group_name(path)
        if group_name:
            group_names.add(group_name)
    return sorted(list(group_names))

def elements_matching_pattern(input_list, pattern):
    return [element for element in input_list if pattern in element]


# --- Phase correlation (edge-based, relative shift) ---
def phase_correlation_alignment(img1, img2, edge_width=250, vertical_region=1000, vertical_offset=300):
    # Grayscale
    if len(img1.shape) == 3:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    else:
        img1_gray = img1.copy()
        img2_gray = img2.copy()

    h1, w1 = img1_gray.shape[:2]
    h2, w2 = img2_gray.shape[:2]

    # Edge ROIs
    y_start1 = min(vertical_offset, max(0, h1 - vertical_region))
    y_start2 = min(vertical_offset, max(0, h2 - vertical_region))
    y_end1 = y_start1 + min(vertical_region, h1 - y_start1)
    y_end2 = y_start2 + min(vertical_region, h2 - y_start2)
    edge_width = min(edge_width, w1, w2)

    img1_edge = img1_gray[y_start1:y_end1, w1 - edge_width:]
    img2_edge = img2_gray[y_start2:y_end2, :edge_width]

    # FFT phase correlation on edge crops
    f1 = fft2(img1_edge.astype(np.float32))
    f2 = fft2(img2_edge.astype(np.float32))
    cps = (f1 * f2.conjugate()) / (np.abs(f1) * np.abs(f2) + 1e-10)
    corr = np.abs(ifft2(cps))
    dy_raw, dx_raw = np.unravel_index(np.argmax(corr), corr.shape)

    # Wrap
    if dy_raw > corr.shape[0] // 2:
        dy_raw -= corr.shape[0]
    if dx_raw > corr.shape[1] // 2:
        dx_raw -= corr.shape[1]

    # Relative shift between the two edge crops:
    dx_rel = -dx_raw
    dy_rel = -dy_raw
    return dx_rel, dy_rel

# --- Feature matching (edge-based, relative shift) ---
def feature_matching_alignment(img1, img2, edge_width=250, vertical_region=1000, vertical_offset=300, feature_method='sift'):
    # Grayscale
    if len(img1.shape) == 3:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    else:
        img1_gray = img1.copy()
        img2_gray = img2.copy()

    h1, w1 = img1_gray.shape[:2]
    h2, w2 = img2_gray.shape[:2]

    # Edge ROIs
    y_start1 = min(vertical_offset, max(0, h1 - vertical_region))
    y_start2 = min(vertical_offset, max(0, h2 - vertical_region))
    y_end1 = y_start1 + min(vertical_region, h1 - y_start1)
    y_end2 = y_start2 + min(vertical_region, h2 - y_start2)
    edge_width = min(edge_width, w1, w2)

    img1_edge = img1_gray[y_start1:y_end1, w1 - edge_width:]
    img2_edge = img2_gray[y_start2:y_end2, :edge_width]

    # Create feature detector based on method
    feature_method = feature_method.lower()
    
    if feature_method == 'sift':
        detector = cv2.SIFT_create()
    elif feature_method == 'orb':
        detector = cv2.ORB_create(nfeatures=5000)
    elif feature_method == 'akaze':
        detector = cv2.AKAZE_create()
    elif feature_method == 'brisk':
        detector = cv2.BRISK_create()
    else:
        print(f"Warning: Unknown feature method '{feature_method}', using SIFT")
        detector = cv2.SIFT_create()

    # Detect features within edge ROIs
    kp1, des1 = detector.detectAndCompute(img1_edge, None)
    kp2, des2 = detector.detectAndCompute(img2_edge, None)

    if des1 is None or des2 is None:
        return 0, 0

    # Create matcher based on descriptor type
    if feature_method == 'orb' or feature_method == 'brisk' or feature_method == 'akaze':
        # Binary descriptors use Hamming distance
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:
        # SIFT uses L2 distance
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    matches = bf.knnMatch(des1, des2, k=2)

    # Lowe ratio test (adjust threshold for binary descriptors)
    ratio_threshold = 0.7 if feature_method in ['orb', 'brisk', 'akaze'] else 0.75
    good = []
    for match_pair in matches:
        if len(match_pair) == 2:
            m, n = match_pair
            if m.distance < ratio_threshold * n.distance:
                good.append(m)

    if len(good) < 4:
        return 0, 0

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)  # img1_edge
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)  # img2_edge

    # Map img2_edge -> img1_edge
    M, mask = cv2.estimateAffinePartial2D(dst_pts, src_pts)
    if M is None:
        return 0, 0

    dx_rel = float(M[0, 2])
    dy_rel = float(M[1, 2])
    return dx_rel, dy_rel

# --- Create distortion map visualization ---
def create_distortion_map(disagreements, output_width, output_height, tile_positions):
    """
    Create a visual distortion map showing where alignment methods disagree most.
    
    Args:
        disagreements: List of (dx_abs, dy_abs, euclidean_distance) for each step
        output_width, output_height: Dimensions of the final stitched image
        tile_positions: List of approximate tile positions in the mosaic
    """
    # Create heatmap canvas
    distortion_map = np.zeros((output_height, output_width, 3), dtype=np.uint8)
    
    # Normalize disagreement values for color mapping
    euclidean_distances = [d[2] for d in disagreements]
    if euclidean_distances:
        max_disagreement = max(euclidean_distances)
        if max_disagreement > 0:
            # Create color map: blue (low disagreement) to red (high disagreement)
            for i, (dx_abs, dy_abs, euclidean) in enumerate(disagreements):
                if i < len(tile_positions):
                    x_pos, y_pos, width, height = tile_positions[i]
                    
                    # Normalize to 0-255 range
                    intensity = min(255, int(255 * euclidean / max_disagreement))
                    
                    # Create color: blue for low disagreement, red for high
                    color = (255 - intensity, 0, intensity)  # BGR format
                    
                    # Draw rectangle representing tile position with disagreement color
                    cv2.rectangle(distortion_map, 
                                (int(x_pos), int(y_pos)), 
                                (int(x_pos + width), int(y_pos + height)), 
                                color, -1)
                    
                    # Add text with disagreement values
                    text = f"{euclidean:.1f}"
                    cv2.putText(distortion_map, text, 
                              (int(x_pos + width//2 - 20), int(y_pos + height//2)), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Add legend
    legend_height = 50
    legend_width = output_width
    legend = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    
    # Create gradient bar
    if euclidean_distances and max(euclidean_distances) > 0:
        gradient = np.linspace(0, 255, legend_width).astype(np.uint8)
        for i in range(legend_width):
            color = (255 - gradient[i], 0, gradient[i])
            legend[:, i] = color
        
        # Add text labels
        cv2.putText(legend, "Low Disagreement", (10, 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(legend, "High Disagreement", (legend_width - 150, 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Combine distortion map and legend
    final_map = np.vstack([distortion_map, legend])
    
    return final_map

# --- Stitch two images with linear blending (same as your working version) ---
def stitch_two_images(img1_path, img2_path, method='phase', add_y_offset=False,
                      edge_width=250, vertical_region=1000, vertical_offset=300, feature_method='sift'):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    if img1 is None or img2 is None:
        raise ValueError(f"Could not load images: {img1_path}, {img2_path}")

    # Choose alignment method
    if method == 'phase':
        dx, dy = phase_correlation_alignment(img1, img2, edge_width=edge_width,
                                             vertical_region=vertical_region, vertical_offset=vertical_offset)
    elif method == 'feature':
        dx, dy = feature_matching_alignment(img1, img2, edge_width=edge_width,
                                            vertical_region=vertical_region, vertical_offset=vertical_offset,
                                            feature_method=feature_method)
    else:
        dx, dy = 0, 0


    # Y sign handling (keep your behavior)
    if not add_y_offset:
        dy = -dy

    # --- Linear blending (unchanged) ---
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    calculated_overlap = edge_width - (-dx - (w2 - edge_width))
    # dx is the shift between edge regions
    # The second image should be positioned at: w1 - edge_width + dx
    img2_x_position = int(round(w1 - edge_width + dx))
    actual_overlap = max(0, w1 - img2_x_position)
    img2_y_position = int(round(dy))

    if img2_y_position >= 0:
        img1_y_start = 0
        img2_y_start = img2_y_position
        output_height = max(h1, h2 + img2_y_position)
    else:
        img1_y_start = -img2_y_position
        img2_y_start = 0
        output_height = max(h1 - img2_y_position, h2)

    output_width = img2_x_position + w2
    stitched = np.zeros((output_height, output_width, 3), dtype=np.uint8)
    stitched[img1_y_start:img1_y_start + h1, :w1] = img1

    if actual_overlap > 0:
        overlap_start = img2_x_position
        overlap_end = w1
        overlap_width = overlap_end - overlap_start
        blend_y_start = max(img1_y_start, img2_y_start)
        blend_y_end = min(img1_y_start + h1, img2_y_start + h2)
        if blend_y_end > blend_y_start and overlap_width > 0:
            blend_height = blend_y_end - blend_y_start
            alpha = np.linspace(1.0, 0.0, overlap_width).reshape(1, -1, 1)
            img1_region = img1[blend_y_start - img1_y_start:blend_y_start - img1_y_start + blend_height,
                               overlap_start:overlap_end]
            img2_region = img2[blend_y_start - img2_y_start:blend_y_start - img2_y_start + blend_height,
                               :overlap_width]
            min_h = min(img1_region.shape[0], img2_region.shape[0])
            min_w = min(img1_region.shape[1], img2_region.shape[1])
            if min_h > 0 and min_w > 0:
                img1_region = img1_region[:min_h, :min_w]
                img2_region = img2_region[:min_h, :min_w]
                alpha = alpha[:, :min_w, :]
                blended = (img1_region.astype(np.float32) * alpha +
                           img2_region.astype(np.float32) * (1 - alpha)).astype(np.uint8)
                stitched[blend_y_start:blend_y_start + min_h,
                         overlap_start:overlap_start + min_w] = blended

    non_overlap_start = w1
    if non_overlap_start < output_width:
        img2_src_x = actual_overlap
        copy_width = w2 - actual_overlap
        if copy_width > 0:
            stitched[img2_y_start:img2_y_start + h2,
                     non_overlap_start:non_overlap_start + copy_width] = \
                img2[:, img2_src_x:img2_src_x + copy_width]

    return stitched

# --- Sequential stitching (phase OR feature) ---
def stitch_multiple_images(image_paths, method='phase', add_y_offset=False,
                           edge_width=250, vertical_region=1000, vertical_offset=300, feature_method='sift'):
    if len(image_paths) < 2:
        return cv2.imread(image_paths[0]) if len(image_paths) == 1 else None

    result = stitch_two_images(image_paths[0], image_paths[1], method=method, add_y_offset=add_y_offset,
                               edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset,
                               feature_method=feature_method)
    for i in range(2, len(image_paths)):
        temp_path = "temp_intermediate.png"
        cv2.imwrite(temp_path, result)
        try:
            result = stitch_two_images(temp_path, image_paths[i], method=method, add_y_offset=add_y_offset,
                                       edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset,
                                       feature_method=feature_method)
        finally:
            if os.path.exists(temp_path):
                os.remove(temp_path)
    return result

# --- Sequential stitching (COMPARE): build both mosaics + print metrics ---
def stitch_multiple_images_compare(image_paths, add_y_offset=False,
                                   edge_width=250, vertical_region=1000, vertical_offset=300, feature_method='sift'):
    if len(image_paths) < 2:
        single = cv2.imread(image_paths[0]) if len(image_paths) == 1 else None
        return single, single, 0.0, 0.0, 0.0, None

    # Start both mosaics
    result_phase = stitch_two_images(image_paths[0], image_paths[1], method='phase', add_y_offset=add_y_offset,
                                     edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset)
    result_feat = stitch_two_images(image_paths[0], image_paths[1], method='feature', add_y_offset=add_y_offset,
                                    edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset,
                                    feature_method=feature_method)

    # Metrics arrays and tile position tracking
    dx_abs_list, dy_abs_list, euclid_list = [], [], []
    disagreements = []
    tile_positions = []
    
    # Get initial tile dimensions for position estimation
    first_img = cv2.imread(image_paths[0])
    tile_height, tile_width = first_img.shape[:2]
    
    # Step 1 metrics on the SAME raw tiles
    img_prev = cv2.imread(image_paths[0])
    img_curr = cv2.imread(image_paths[1])
    dx_p, dy_p = phase_correlation_alignment(img_prev, img_curr, edge_width=edge_width,
                                             vertical_region=vertical_region, vertical_offset=vertical_offset)
    dx_f, dy_f = feature_matching_alignment(img_prev, img_curr, edge_width=edge_width,
                                            vertical_region=vertical_region, vertical_offset=vertical_offset,
                                            feature_method=feature_method)

    dx_p  = -1*dx_p # align sign to dx_f
    dy_p  = -1*dy_p # align sign to dy_f
    dx_abs = abs(dx_p - dx_f)
    dy_abs = abs(dy_p - dy_f)
  
    euclid = np.sqrt((dx_p - dx_f)**2 + (dy_p - dy_f)**2)
    dx_abs_list.append(dx_abs); dy_abs_list.append(dy_abs); euclid_list.append(euclid)
    disagreements.append((dx_abs, dy_abs, euclid))
    
    # Estimate first tile position (approximately where the overlap occurs)
    tile_positions.append((tile_width - edge_width, 0, edge_width, tile_height))
    
    print(f"Step 1 disagreement ({feature_method.upper()}): dx_abs={dx_abs:.2f}, dy_abs={dy_abs:.2f}, euclid={euclid:.2f}")    
    print("dx_phase: ",dx_p)
    print("dx_feature: ",round(dx_f,2))
    print("dy_phase: ",dy_p)
    print("dy_feature: ",round(dy_f,2),"\n")

    # Continue both mosaics; metrics always on raw consecutive tiles (same pair basis)
    current_x_pos = tile_width
    for i in range(2, len(image_paths)):
        # Update mosaics
        temp_phase = "temp_phase.png"
        temp_feat = "temp_feat.png"
        cv2.imwrite(temp_phase, result_phase)
        cv2.imwrite(temp_feat, result_feat)
        try:
            result_phase = stitch_two_images(temp_phase, image_paths[i], method='phase', add_y_offset=add_y_offset,
                                             edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset)
            result_feat = stitch_two_images(temp_feat, image_paths[i], method='feature', add_y_offset=add_y_offset,
                                            edge_width=edge_width, vertical_region=vertical_region, vertical_offset=vertical_offset,
                                            feature_method=feature_method)
        finally:
            if os.path.exists(temp_phase): os.remove(temp_phase)
            if os.path.exists(temp_feat): os.remove(temp_feat)

        # Metrics on raw tiles (i-1 vs i), same basis
        img_prev = cv2.imread(image_paths[i-1])
        img_curr = cv2.imread(image_paths[i])
        dx_p, dy_p = phase_correlation_alignment(img_prev, img_curr, edge_width=edge_width,
                                                 vertical_region=vertical_region, vertical_offset=vertical_offset)
        dx_f, dy_f = feature_matching_alignment(img_prev, img_curr, edge_width=edge_width,
                                                vertical_region=vertical_region, vertical_offset=vertical_offset,
                                                feature_method=feature_method)
        
        dx_p  = -1*dx_p # align sign to dx_f
        dy_p  = -1*dy_p # align sign to dy_f
        dx_abs = abs(dx_p - dx_f)
        dy_abs = abs(dy_p - dy_f)
    
        euclid = np.sqrt((dx_p - dx_f)**2 + (dy_p - dy_f)**2)
        dx_abs_list.append(dx_abs); dy_abs_list.append(dy_abs); euclid_list.append(euclid)
        disagreements.append((dx_abs, dy_abs, euclid))
        
        # Estimate tile position (simplified - assumes tiles are added horizontally)
        tile_positions.append((current_x_pos - edge_width, 0, edge_width, tile_height))
        current_x_pos += (tile_width - edge_width)  # Approximate next position
        
        print(f"Step {i} disagreement ({feature_method.upper()}): dx_abs={dx_abs:.2f}, dy_abs={dy_abs:.2f}, euclid={euclid:.2f}")
        print("dx_phase: ",dx_p)
        print("dx_feature: ",round(dx_f,2))
        print("dy_phase: ",dy_p)
        print("dy_feature: ",round(dy_f,2),"\n")

    # Overall means
    mean_dx_abs = float(np.mean(dx_abs_list)) if dx_abs_list else 0.0
    mean_dy_abs = float(np.mean(dy_abs_list)) if dy_abs_list else 0.0
    mean_euclid = float(np.mean(euclid_list)) if euclid_list else 0.0
    
    print(f"\nOverall mean abs disagreement ({feature_method.upper()}): dx={mean_dx_abs:.2f}, dy={mean_dy_abs:.2f}, euclid={mean_euclid:.2f},\n")

    
    # Create distortion map
    if result_phase is not None:
        output_height, output_width = result_phase.shape[:2]
        distortion_map = create_distortion_map(disagreements, output_width, output_height, tile_positions)
    else:
        distortion_map = None

    return result_phase, result_feat, mean_dx_abs, mean_dy_abs, mean_euclid, distortion_map





In [51]:
# --- Main ---
year = '2020'
Session_number = '03'
Season = 'November'

Session = f'Session{Session_number}'
directory_path = f'C:/Users/jocu0013/Desktop/Oulanka/Scan_Raw/Oulanka{year}_{Season}/'
path_out = f'D:/Blending_Scans/test/{year}_{Session_number}/Oulanka{year}_{Session}_'
pattern = '.tiff'

method = 'compare'  # 'phase', 'feature', or 'compare'
feature_method = 'orb'  # 'sift', 'orb', 'akaze', 'brisk' (ignored if method is not 'feature' or 'compare')
add_y_offset = True  # (debugging, I think it should always be True, needs clean up)

# Define subsets with their specific parameters (usefull if images differ in overlap or other properties)
# * 'name': Descriptive name for this subset (used in output messages)
# * 'range': Index range in the image list (start, end) using Python slice notation
# * 'params':
#   -> 'edge_width': Width in pixels of edge regions for alignment. Use larger values (400-800) 
#     for low overlap (~100-200px), smaller values (150-300) for high overlap (>300px)
#   -> 'vertical_region': Height in pixels of vertical strip for alignment (typical: 500-2000)
#   -> 'vertical_offset': Starting Y position for vertical region (0=top, larger=skip top portion)

# .... for me, forest scans have high and variable overlap, fen images have low overlap ca. 110px 
# reducing edge_width close to actual overlap * 1-2 improves alignment 
subsets = [
#    {
#        'name': 'forest',
#        'range': (0, 48),
#        'params': {
#            'edge_width': 1600, # more for high overlap (must be <image width)
#            'vertical_region': 1200,
#            'vertical_offset': 300,
#        }
#    },
    {
        'name': 'fen',
        'range': (0, 212),
        'params': {
            'edge_width': 250, # low overlap (actual overlap is ~0.5 edge_width)
            'vertical_region': 1200,
            'vertical_offset': 300,
        }
    },
   
    # Add more subsets as needed
    # {
    #     'name': 'meadow',
    #     'range': (274, 400),
    #     'params': {
    #         'edge_width': 300,
    #         'vertical_region': 1000,
    #         'vertical_offset': 300,
    #     }
    # }
]

def process_subset(imgs, correlation_params, subset_name):
    """Process a subset of images with specific parameters"""
    unique_names = get_unique_group_names(imgs)
    
    print(f"\n{'='*60}")
    print(f"Processing {subset_name.upper()} subset")
    print(f"Found {len(unique_names)} unique image sets")
    print(f"Method: {method}")
    if method in ['feature', 'compare']:
        print(f"Feature method: {feature_method.upper()}")
    print(f"Parameters: {correlation_params}")
    print(f"{'='*60}\n")
    
    for i, unique_name in enumerate(unique_names):
        print(f"\nProcessing {i+1}/{len(unique_names)}: {unique_name}")
        matching_images = [img for img in imgs if extract_group_name(img) == unique_name]
        matching_images.sort()
        
        if len(matching_images) == 1:
            img = cv2.imread(matching_images[0])
            cv2.imwrite(path_out + str(unique_name) + ".png", img)
            print("  Single image - copied")
            continue
        
        try:
            if method == 'compare':
                stitched_phase, stitched_feat, mean_dx, mean_dy, mean_e, distortion_map = \
                    stitch_multiple_images_compare(matching_images, add_y_offset=add_y_offset, 
                                                   feature_method=feature_method, **correlation_params)
                cv2.imwrite(path_out + str(unique_name) + "_phase.png", stitched_phase)
                cv2.imwrite(path_out + str(unique_name) + f"_{feature_method}.png", stitched_feat)
                
                # Save distortion map if created
                if distortion_map is not None:
                    cv2.imwrite(path_out + str(unique_name) + "_distortion_map.png", distortion_map)
                    print(f"  Distortion map saved")
                
                print(f"  {len(matching_images)} images stitched (phase & {feature_method}). "
                      f"Mean |Δdx|={mean_dx:.2f}, |Δdy|={mean_dy:.2f}, euclid={mean_e:.2f}")
                print("#################################\n")
            else:
                stitched = stitch_multiple_images(matching_images, method=method, add_y_offset=add_y_offset, 
                                                  feature_method=feature_method, **correlation_params)
                suffix = f"_{feature_method}" if method == 'feature' else ""
                cv2.imwrite(path_out + str(unique_name) + suffix + ".png", stitched)
                method_name = f"{method} ({feature_method})" if method == 'feature' else method
                print(f"  {len(matching_images)} images stitched using method '{method_name}'")
                
        except Exception as e:
            print(f"  ERROR processing {unique_name}: {str(e)}")

def main():
    imgs = list_files_with_pattern(directory_path, pattern)
    os.makedirs(os.path.dirname(path_out), exist_ok=True)
    
    # --- NEW: Count total and per-subset unique image sets ---
    total_unique = set()
    subset_summary = []
    
    for subset in subsets:
        imgs_subset = imgs[subset['range'][0]:subset['range'][1]]
        if len(imgs_subset) == 0:
            subset_summary.append((subset['name'], 0))
            continue

        unique_names = get_unique_group_names(imgs_subset)
        total_unique.update(unique_names)
        subset_summary.append((subset['name'], len(unique_names)))

    print("\n" + "="*70)
    print(f"📂 Total unique image sets found: {len(total_unique)}")
    for name, count in subset_summary:
        print(f"   - {count} in subset '{name}'")
    print("="*70 + "\n")
    
    
    # Process each subset with its specific parameters
    for subset in subsets:
        imgs_subset = imgs[subset['range'][0]:subset['range'][1]]
        if len(imgs_subset) > 0:
            process_subset(imgs_subset, subset['params'], subset['name'])
        else:
            print(f"\nSkipping {subset['name']} - no images in range {subset['range']}")

if __name__ == "__main__":
    main()


📂 Total unique image sets found: 36
   - 36 in subset 'fen'


Processing FEN subset
Found 36 unique image sets
Method: compare
Feature method: ORB
Parameters: {'edge_width': 250, 'vertical_region': 1200, 'vertical_offset': 300}


Processing 1/36: T037
Step 1 disagreement (ORB): dx_abs=0.01, dy_abs=0.01, euclid=0.01
dx_phase:  124
dx_feature:  123.99
dy_phase:  -1
dy_feature:  -0.99 

Step 2 disagreement (ORB): dx_abs=0.01, dy_abs=1.52, euclid=1.52
dx_phase:  109
dx_feature:  109.01
dy_phase:  -5
dy_feature:  -3.48 

Step 3 disagreement (ORB): dx_abs=0.38, dy_abs=0.29, euclid=0.48
dx_phase:  103
dx_feature:  103.38
dy_phase:  6
dy_feature:  6.29 

Step 4 disagreement (ORB): dx_abs=0.00, dy_abs=0.59, euclid=0.59
dx_phase:  105
dx_feature:  105.0
dy_phase:  -6
dy_feature:  -5.41 

Step 5 disagreement (ORB): dx_abs=108.00, dy_abs=7.00, euclid=108.23
dx_phase:  108
dx_feature:  0
dy_phase:  -7
dy_feature:  0 


Overall mean abs disagreement (ORB): dx=21.68, dy=1.88, euclid=22.17,

  Distor

  Distortion map saved
  5 images stitched (phase & orb). Mean |Δdx|=0.63, |Δdy|=2.60, euclid=2.79
#################################


Processing 10/36: T046
Step 1 disagreement (ORB): dx_abs=0.84, dy_abs=0.23, euclid=0.87
dx_phase:  116
dx_feature:  116.84
dy_phase:  -8
dy_feature:  -7.77 

Step 2 disagreement (ORB): dx_abs=0.17, dy_abs=0.33, euclid=0.37
dx_phase:  109
dx_feature:  108.83
dy_phase:  2
dy_feature:  2.33 

Step 3 disagreement (ORB): dx_abs=0.55, dy_abs=1.37, euclid=1.48
dx_phase:  104
dx_feature:  104.55
dy_phase:  -8
dy_feature:  -6.63 

Step 4 disagreement (ORB): dx_abs=0.37, dy_abs=0.66, euclid=0.76
dx_phase:  108
dx_feature:  108.37
dy_phase:  6
dy_feature:  6.66 

Step 5 disagreement (ORB): dx_abs=0.17, dy_abs=0.98, euclid=1.00
dx_phase:  105
dx_feature:  104.83
dy_phase:  -4
dy_feature:  -3.02 


Overall mean abs disagreement (ORB): dx=0.42, dy=0.71, euclid=0.89,

  Distortion map saved
  6 images stitched (phase & orb). Mean |Δdx|=0.42, |Δdy|=0.71, euclid=0.89
##

Step 1 disagreement (ORB): dx_abs=249.70, dy_abs=2.91, euclid=249.72
dx_phase:  -116
dx_feature:  133.7
dy_phase:  12
dy_feature:  14.91 

Step 2 disagreement (ORB): dx_abs=0.43, dy_abs=0.50, euclid=0.66
dx_phase:  106
dx_feature:  105.57
dy_phase:  -1
dy_feature:  -0.5 

Step 3 disagreement (ORB): dx_abs=0.19, dy_abs=0.28, euclid=0.34
dx_phase:  107
dx_feature:  107.19
dy_phase:  1
dy_feature:  1.28 

Step 4 disagreement (ORB): dx_abs=0.25, dy_abs=0.50, euclid=0.56
dx_phase:  102
dx_feature:  101.75
dy_phase:  -2
dy_feature:  -2.5 

Step 5 disagreement (ORB): dx_abs=0.18, dy_abs=0.07, euclid=0.19
dx_phase:  106
dx_feature:  106.18
dy_phase:  -4
dy_feature:  -4.07 


Overall mean abs disagreement (ORB): dx=50.15, dy=0.85, euclid=50.29,

  Distortion map saved
  6 images stitched (phase & orb). Mean |Δdx|=50.15, |Δdy|=0.85, euclid=50.29
#################################


Processing 20/36: T056
Step 1 disagreement (ORB): dx_abs=1.16, dy_abs=0.38, euclid=1.22
dx_phase:  121
dx_feature:  

Step 5 disagreement (ORB): dx_abs=0.11, dy_abs=0.21, euclid=0.24
dx_phase:  102
dx_feature:  101.89
dy_phase:  -4
dy_feature:  -4.21 


Overall mean abs disagreement (ORB): dx=50.50, dy=1.51, euclid=50.93,

  Distortion map saved
  6 images stitched (phase & orb). Mean |Δdx|=50.50, |Δdy|=1.51, euclid=50.93
#################################


Processing 29/36: T065
Step 1 disagreement (ORB): dx_abs=0.04, dy_abs=1.36, euclid=1.36
dx_phase:  125
dx_feature:  125.04
dy_phase:  6
dy_feature:  7.36 

Step 2 disagreement (ORB): dx_abs=0.16, dy_abs=0.50, euclid=0.53
dx_phase:  105
dx_feature:  104.84
dy_phase:  1
dy_feature:  1.5 

Step 3 disagreement (ORB): dx_abs=0.10, dy_abs=0.12, euclid=0.16
dx_phase:  106
dx_feature:  106.1
dy_phase:  -3
dy_feature:  -2.88 

Step 4 disagreement (ORB): dx_abs=0.01, dy_abs=0.18, euclid=0.18
dx_phase:  108
dx_feature:  108.01
dy_phase:  2
dy_feature:  2.18 

Step 5 disagreement (ORB): dx_abs=0.06, dy_abs=0.34, euclid=0.35
dx_phase:  105
dx_feature:  104.94
d